#import

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import KFold
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor, Pool
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
#import cupy as cp
import tensorflow as tf
import shap
import optuna
import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Load files
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
ss = pd.read_csv('SampleSubmission.csv')

#handeling

In [3]:
fe = []; cat_fe = []; no_fe = []; three_cat_columns = []
for i in train.columns[1:]:
  if train[i].isnull().sum() < 5000:
    fe.append(i)
    if train[i].dtype == 'O':
      cat_fe.append(i)
      print("There are "+ str(len(train[i].value_counts())) + " in " + str(i))
      if len(train[i].unique()) == 3:
          three_cat_columns.append(i)
  else:
    no_fe.append(i) 

print ('----------------------------------')
print(str(len(fe)))
print(str(len(cat_fe)))
print(str(len(no_fe)))
print(str(len(three_cat_columns)))

There are 254 in child_date
There are 4 in child_grant
There are 4 in child_years_in_programme
There are 4 in child_observe_attentive
There are 4 in child_observe_concentrated
There are 4 in child_observe_diligent
There are 4 in child_observe_interested
There are 2 in child_gender
There are 1018 in child_dob
There are 3 in child_stunted
There are 4 in child_age_group
There are 153 in id_mn_best
There are 10 in prov_best
There are 50 in id_dc_best
There are 50 in dc_best
There are 153 in mn_best
There are 5 in pra_free_play
There are 4 in pra_free_play_outdoor
There are 31 in pra_groupings
There are 2 in pra_groupings_1
There are 2 in pra_groupings_2
There are 2 in pra_groupings_3
There are 2 in pra_groupings_4
There are 2 in pra_groupings_5
There are 3 in pra_engaged
There are 4 in pra_agency_choice
There are 4 in pra_agency_explore
There are 4 in pra_agency_questions
There are 4 in pra_agency_understand
There are 4 in pra_agency_play
There are 4 in pra_agency_learn
There are 4 in pra_

In [4]:
fet = fe.copy()
fet.remove('target')

#catagorical

In [5]:
tra = train[fe]
te = test[fet]

handeling 

In [6]:
x = tra.drop(columns=['target'])
y = tra['target']
x = x.fillna('')
y = y.fillna('')
te = te.fillna('')

Spliting train and test data

In [7]:
# Split data into training and test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42, shuffle = True)

In [8]:
train_dataset = Pool(data = X_train, label = y_train, cat_features=cat_fe)
test_dataset = Pool(data = X_test, label = y_test, cat_features=cat_fe)

MODELS TO TRAIN

In [9]:
params = {'learning_rate': 0.034635131965162685, 'l2_leaf_reg': 9.45201367157322, 'grow_policy': 'Lossguide', 'bootstrap_type': 'Bayesian'}
cmod = CatBoostRegressor(iterations = 30000,
                                   learning_rate=0.1,
                                  random_seed = 42, verbose = 300)
cmod.fit(train_dataset, eval_set = test_dataset, early_stopping_rounds = 300, use_best_model = True)

0:	learn: 14.6697892	test: 14.8425844	best: 14.8425844 (0)	total: 668ms	remaining: 5h 34m 7s
300:	learn: 8.9776987	test: 10.2001134	best: 10.1998195 (299)	total: 2m 18s	remaining: 3h 47m 23s
600:	learn: 8.1456023	test: 10.0047198	best: 10.0043380 (598)	total: 5m 15s	remaining: 4h 16m 58s


KeyboardInterrupt: 

In [ ]:
predc = cmod.predict(te)
testc = cmod.predict(X_test)
mean_squared_error(y_test, testc, squared=False)

9.893919207982812

#submission

In [ ]:
expa = shap.TreeExplainer(cmod)
shap_val = expa.shap_values(te)

In [34]:
features = te.columns

In [35]:
f = {
    'f1': [],'f2' : [],'f3' : [],'f4' : [],'f5' : [],'f6' : [],'f7' : [],'f8' : [],'f9' : [],'f10' : [],'f11' : [],'f12' : [],'f13' : [],'f14': [],'f15' : []
}

In [36]:
for shap_vals in shap_val:
  arr = np.argsort(shap_vals)[::-1][:15]
  for ind, a in enumerate(arr):
    name_f = f'f{ind+1}'
    f[name_f].append(features[a])

In [37]:
ss['target'] = predl
ss['feature_1'] = f['f1']
ss['feature_2'] = f['f2']
ss['feature_3'] = f['f3']
ss['feature_4'] = f['f4']
ss['feature_5'] = f['f5']
ss['feature_6'] = f['f6']
ss['feature_7'] = f['f7']
ss['feature_8'] = f['f8']
ss['feature_9'] = f['f9']
ss['feature_10'] = f['f10']
ss['feature_11'] = f['f11']
ss['feature_12'] = f['f12']
ss['feature_13'] = f['f13']
ss['feature_14'] = f['f14']
ss['feature_16'] = f['f15']

In [38]:
ss

,child_id,target,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16
0,ID_0I0999N6S,59.435389,child_observe_total,teacher_emotional_total,count_staff_qual_skills,teacher_emotional_met_Meets the standard,count_children_present,pri_fees_amount,pri_fees_amount_pv,count_toilets_children,ses_proxy,id_team,longitude,teacher_emotional_met_Does not meet standard,id_mn_n,count_register_year_grader,feature,pra_groupings_2_Yes
1,ID_GQ6ONJ4FP,43.535324,child_observe_total,teacher_emotional_total,teacher_emotional_met_Meets the standard,pri_fees_amount,count_children_present,pri_year,teacher_emotional_met_Does not meet standard,count_register_year_grader,count_register_year_2020,count_children_precovid,count_staff_salary_unpaid,count_register_year_2015,id_facility,child_months_enrolment,feature,teacher_selfcare_met_Does not meet standard
2,ID_YZ76CVRW3,48.239335,child_observe_total,id_enumerator,id_team,teacher_selfcare_total,latitude,id_prov_n,obs_space_No,pri_year,pri_fees_amount_pv,ses_proxy,obs_materials_5_No,pri_fees_amount,count_register_year_2019,count_register_year_2020,feature,teacher_selfcare_met_nan
3,ID_BNINCRXH8,67.868224,child_observe_total,id_enumerator,longitude,teacher_duration,pra_ind_Yes,id_team,count_children_present,pri_capacity,teacher_emotional_met_Does not meet standard,child_age,count_children_attendance,count_register_gender,quintile,teacher_emotional_met_Meets the standard,feature,quintile_used_nan
4,ID_1U7GDTLRI,43.939194,child_observe_total,id_ward,id_facility,latitude,longitude,id_team,pri_meal_1_No,count_register_race_coloured,urban_Urban,count_staff_qual_skills,pri_time_close_hours,obs_materials_8_No,child_months_enrolment,count_register_year_2020,feature,pri_registered_npo_Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3675,ID_LBPQ2VMQZ,51.980638,child_age,child_observe_total,ses_proxy,id_ward,pra_ind_Yes,id_team,teacher_selfcare_total,teacher_emotional_met_Does not meet standard,data_year,count_children_attendance,count_register_year_2020,pri_registered_npo_Yes,teacher_selfcare_met_Does not meet standard,pri_holidays_No,feature,pri_funding_subsidy
3676,ID_H2RKKMMKK,47.798288,child_age,id_ward,teacher_selfcare_total,id_team,pri_fees_amount,id_enumerator,child_zha,teacher_social_total,obs_handwashing_1_No,child_height,longitude,pri_fees_amount_pv,count_children_present,teacher_selfcare_met_nan,feature,count_register_year_2020
3677,ID_VY8KX7YTZ,37.103089,child_observe_total,teacher_selfcare_total,obs_space_No,longitude,teacher_emotional_met_Does not meet standard,count_register_year_2018,pri_expense_food,teacher_social_total,count_register_year_2020,teacher_selfcare_met_Does not meet standard,obs_materials_6_No,teacher_selfcare_met_nan,obs_materials_11_Yes,obs_materials_10_No,feature,pra_hhsize
3678,ID_EO2MYZ4M7,44.452384,id_enumerator,id_facility,pri_capacity,obs_heating_1,child_height,obs_space_No,latitude,count_children_present,pqa_score_assessment,pri_expense_admin,child_zha,count_practitioners_age_5,obs_handwashing_1_No,pri_meal_2_No,feature,obs_materials_8_No


In [39]:
ss.to_csv('BaselineSubmission83.csv', index=False)